In [145]:
# Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [126]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

In [127]:
wiki.name

'wikipedia'

In [128]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [129]:
arxiv.name

'arxiv'

In [130]:
tools=[wiki,arxiv]

In [131]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\LLM Youtube Video Summarizer\\venvyvs\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [132]:
## using tools with agents and llama

In [146]:
# Run all this tools with Agents and LLM Models

# Tools, LLM-->AgentExecutor
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()

groq_api_key = os.getenv("groq_api_key")
openai.api_key = os.getenv("openai_api_key")

llm = ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [147]:
llm.invoke("Hi, I'm Mayank")

AIMessage(content="Hello Mayank! It's nice to meet you. Is there something I can help you with or would you like to chat?", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_time': 0.0225, 'prompt_time': 0.001984789, 'queue_time': 0.011703908999999998, 'total_time': 0.024484789}, 'model_name': 'Llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c34dcb7-799b-40d0-8364-5b6646017273-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43})

In [148]:
# Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

d:\LLM Youtube Video Summarizer\venvyvs\lib\site-packages\langsmith\client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [149]:
# Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_has

In [150]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [151]:
agent_executor.invoke({"input": "Tell me about Cricket sports"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'cricket'}`


Page: Cricket
Summary: Cricket is a bat-and-ball game that is played between two teams of eleven players on a field, at the centre of which is a 22-yard (20-metre; 66-foot) pitch with a wicket at each end, each comprising two bails balanced on three 
Invoking: `wikipedia` with `{'query': 'cricket_rules'}`


Page: Laws of Cricket
Summary: The Laws of Cricket is a code that specifies the rules of the game of cricket worldwide. The earliest known code was drafted in 1744. Since 1788, the code has been owned and maintained by the private Marylebone Cricket Based on the information provided by the tools, I can summarize the key points about cricket as follows:

* Cricket is a bat-and-ball game played between two teams of eleven players on a field with a wicket at each end.
* The game is played with a ball and a bat, and the objective is to score runs by hitting the ball and running between the wicke

{'input': 'Tell me about Cricket sports',
 'output': 'Based on the information provided by the tools, I can summarize the key points about cricket as follows:\n\n* Cricket is a bat-and-ball game played between two teams of eleven players on a field with a wicket at each end.\n* The game is played with a ball and a bat, and the objective is to score runs by hitting the ball and running between the wickets.\n* The Laws of Cricket specify the rules of the game, which have been drafted and maintained by the Marylebone Cricket Club since 1788.\n* The game is played with a ball and a bat, and the objective is to score runs by hitting the ball and running between the wickets.\n\nI hope this summary is helpful! Let me know if you have any further questions.'}

In [152]:
[1706.03762]

[1706.03762]

In [153]:
agent_executor.invoke({"input": "Tell me about 1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cAttention Is All You Need is a famous paper that introduced the Transformer model, which revolutionized the field of Natural Language Processing (NLP). Instead of relying on Recurrent Neural Networks (RNNs) or Convolutional Neural Networks (CNNs), the Transformer uses self-attention mechanisms to process input sequences. This approach has since become a 

{'input': 'Tell me about 1706.03762',
 'output': 'Attention Is All You Need is a famous paper that introduced the Transformer model, which revolutionized the field of Natural Language Processing (NLP). Instead of relying on Recurrent Neural Networks (RNNs) or Convolutional Neural Networks (CNNs), the Transformer uses self-attention mechanisms to process input sequences. This approach has since become a cornerstone of many state-of-the-art NLP models.'}